
# Jupyter notebook for the case study (using Python 3)

## Task 1

**_1) Setup_**

importing libraries to process data. Pandas package to work with Dataframes. Numpay package for math / linear algebra.

In [1]:
import pandas as pd
import numpy as np

defining dataset names. Can change names to add other datasets.

In [2]:
#uncomment for testing with small datasets
name_dataset_0 = 'small_app_dataset.csv' # 'app_dataset.csv'
name_dataset_1 = 'small_dataset_1.csv' # 'dataset_1.csv'
name_dataset_2 = 'small_dataset_2.csv' # 'dataset_2.csv'

In [3]:
name_dataset_0 = 'app_dataset.csv'
name_dataset_1 = 'dataset_1.csv'
name_dataset_2 = 'dataset_2.csv'

defining key names

In [4]:
key1 = 'key1'
key2 = 'key2'
key_names = [key1, key2]

saving CSV fomratted datasets as Pandas dataframes

In [5]:
dataset_0 = pd.read_csv(name_dataset_0, sep=';')
dataset_1 = pd.read_csv(name_dataset_1, sep=';')
dataset_2 = pd.read_csv(name_dataset_2, sep=';')

**_2) Investigating the datasets - checking how many rows, columns and elements they have_**

function to print the number of columns, rows and elements for each dataset

In [6]:
def print_col_row_and_cell_count(df):
    row_count, column_count = df.shape
    element_count = column_count*row_count
    print('column count:  ', column_count)
    print('row count:     ', row_count)
    print('element count: ', element_count)
    print()

total number of row and column count for each dataset (including NA values)

In [7]:
print('1) dataset 0')
print_col_row_and_cell_count(dataset_0)
print('2) dataset 1')
print_col_row_and_cell_count(dataset_1)
print('3) dataset 2')
print_col_row_and_cell_count(dataset_2)

1) dataset 0
column count:   5
row count:      798
element count:  3990

2) dataset 1
column count:   169
row count:      14571
element count:  2462499

3) dataset 2
column count:   37
row count:      10137
element count:  375069



**_3) Joining the datasets_**

In [8]:
dataset_0_and_1 = pd.merge(dataset_0, dataset_1, how='left', on=key2)

In [9]:
dataset_full_not_cleaned = pd.merge(dataset_0_and_1, dataset_2, how='left', on=key1)

In [10]:
dataset_full_not_cleaned.to_csv('output_dataset_full_not_cleaned.csv')

In [11]:
print('dataset_full - before cleaning NAs')
print_col_row_and_cell_count(dataset_full_not_cleaned)

dataset_full - before cleaning NAs
column count:   209
row count:      798
element count:  166782



**_4) Dropping columns with keysl.Removing columns and rows containing many NA values. Saving the final dateset to CSV file_**

After the join is done, keys are not needed. Dropping them.

In [130]:
dataset_full_not_cleaned_keys_dropped = dataset_full_not_cleaned.drop(key_names, axis=1)

Function to deal with NA values. It will drop rows and columns if the amount of non-NA values in a given column or row is below a given threshold. By default it is 20% for columns and 5% for rows.

In [123]:
def drop_rows_and_cols_with_NA_below_thresholds(input_df, key_names=key_names, col_thresh=0.20, row_thresh=0.05):
    df = input_df.copy(deep=True)
    
    number_of_cols = len(list(df.columns))
    row_threshold_integer = round(row_thresh * number_of_cols)
    df = df.dropna(axis=0, thresh=row_threshold_integer) # droping rows that have non-NA cell count below threshold
    
    number_of_rows = len(df)
    col_threshold_integer = round(col_thresh * number_of_rows)
    df = df.dropna(axis=1, thresh=col_threshold_integer) # droping columns that have non-NA cell count below threshold
    return df

In [131]:
dataset_full_clean = drop_rows_and_cols_with_NA_below_thresholds(dataset_full_not_cleaned_keys_dropped, 
                                                                 col_thresh=0.20, row_thresh=0.05)

In [125]:
print('dataset_full_clean - after some columns and rows with many missing values are removed')
print_col_row_and_cell_count(dataset_full_clean)

dataset_full_clean - after some columns and rows with many missing values are removed
column count:   60
row count:      772
element count:  46320



Saving the final dataset as a CSV file

In [126]:
dataset_full_clean.to_csv('output_dataset_full_clean.csv')

**_5) Observations on data integrity _**

Overall, we see that a lot of data is not used. In the final table we have 798 rows (the same as in the 'master' dataset_0, because that dataset is used in left outer join). Dataset1 has 14571 rows, and dataset2 - 10137. Since response variable is available only for these 798 rows, we have to ignore most of the rows from dataset1 and dataset2. 

On top of that, there are a lot of missing values (NA), especially in the dataset1. The combined dataset has 209 columns, before the columns with many NAs are removed. After I remove them, applying 20% threshold, only 62 columns remain. [UPDATE - provide counts on NA in each table. Maybe update print function to show NA cells as well]

**_3) .....handling NA in some other way???...... _**

## Task 2

**_1) Setup_**

(a) Defining the name of the target variable

In [16]:
target = 'response'

(b) Defining function to get all column names except for the target and key columns. Will allow to dynamically analyze dataframes without the need to know exact columns they have

In [17]:
def get_col_names_without_target_and_keys(dataframe, key_names = key_names, target = target):
    all_column_names_set = set(dataframe)
    col_names_without_target_and_keys = all_column_names_set - set(key_names) - set([target])
    return list(col_names_without_target_and_keys)

(c) We want to determine which factors are the most important in predicting target variable (response). Many variables still has too many NAs, so I will use more agressive column threshold (60%) to remove columns/factors with many missing values. Otherwise, we would introduce too much bias if we would try to impute them all.

In [132]:
dataset_full = drop_rows_and_cols_with_NA_below_thresholds(dataset_full_clean, col_thresh=0.60, row_thresh=0.05)
print_col_row_and_cell_count(dataset_full)

column count:   38
row count:      772
element count:  29336



(d) Python uses '.' as a decimal point. However, in datasets sometimes we get ',' as a decimal point. Need to replace ',' with '.'. The after this is done, will need to convert floats stored as string to Python floats.

In [133]:
def replace_commas_with_dots_in_string(single_string):
    if type(single_string) == str:
        single_string = single_string.replace(',','.')
    return single_string

In [134]:
dataset_full = dataset_full.applymap(replace_commas_with_dots_in_string)

In [135]:
#function to convert floats stored as string to floats
def convert_floats_in_string_to_floats(element):
    if type(element) == str or type(element) == int:
        try:
            return float(element)
        except (ValueError, TypeError):
            return element
    return element

We do not want to convert key and response columns to float, so need to obtain a list of all columns except for keys and target.

In [136]:
columns_to_convert = get_col_names_without_target_and_keys(dataset_full)

Converting all columns except for keys and response.

In [137]:
dataset_full.loc[:,columns_to_convert] = (dataset_full[columns_to_convert]).applymap(convert_floats_in_string_to_floats)

**_2) Imputing remaining missing values_**

There are still many missing values. In order to use Machine Learning models in Task 2 and 3, it is required to get rid of missing values. In the previous I have removed some. The remaining will be imputed.

In [ ]:
dataset_full_np = dataset_full.values

In [ ]:
import Orange

In [ ]:
dataset_full_orange = Orange.data.Table(dataset_full_np)

In [ ]:
take only text columns 

In [ ]:
replace ',' with '.' in floats => use regex (*[N*int','M*int])

impute missing values!!!

normalize all data

one hot encoder / create dummies for text information

train regularized regression 

dimensionality reduction

## Task 4

1) Deal with imbalaced dataset. Out of 798 observations, response variable is 0 in 645 observations, and it is 1 in 153 cases. It is not a very big disbalance, but it is possible that prediction accuracy would be better if I would deal with this imbalancing. (a) The simplest approach is to randomly remove 492 rows where response variable is 0, this would result in a balaced dataset where we have 153 cases of response variable being 0 and 153 casee being 1. (b) A bit better approach would be to put more weight on obseravations where response is 1. Each such observation would weigh 4.2 (645/153). (c) Employ some of the many other approaches of dealing with imbalanced dataset.

2) Columns v173, v175 and v177 contain some date information. It would be good to understand what these dates are about and then to extract some valuable features. It could be: duration, starting and end time in hours, days, months, etc. Such information could be helpful at making better predictions.


3) I am mainly removing columns with many NAs. For rows I was more conservative - I was removing only those that had all NA values except for key columns. It might be beneficial to apply a threshold and remove rows that has too many missing values (similarly as I did with columns).

4) Use better techniques for dimensionality reduction

5) Use SVM for sparse datasets

In [ ]:
6) Drop columns that has too few variations.

In [ ]:
dataset_full_with_dummies = dataset_full.copy(deep=True)

In [ ]:
dataset_full_with_dummies.ge

In [ ]:
dataset_full_np2

In [ ]:
len(dataset_full_np2)

In [ ]:
dataset_full_np2.shape

In [ ]:
dataset_full_np2[0]

In [ ]:
dataset_full_np.shape

In [ ]:
len(dataset_full_np)

In [ ]:
type(dataset_full_np)

In [ ]:
dataset_full_np

In [ ]:
dataset_full

In [ ]:
dataset_0_and_1

In [ ]:
dataset_full

In [ ]:
len(dataset_full)

In [ ]:
dataset_0.shape[1]

In [ ]:
dataset_example = dataset_1.copy(deep=True)
dataset_example.shape

In [ ]:
#dataset_example = dataset_example.dropna(axis=0, how='all', subset=all_columns_no_key)

In [ ]:
#dataset_example

In [ ]:
list(dataset_0)

In [ ]:
dataset_0

In [ ]:
#set(dataset_1)

In [ ]:
dataset_1

In [ ]:
#list(dataset_2)

In [ ]:
dataset_2

In [ ]:
 df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                        'B': ['B0', 'B1', 'B2', 'B3'],
                        'C': ['C0', 'C1', 'C2', 'C3'],
                        'D': ['D0', 'D1', 'D2', 'D3']},  index=[0, 1, 2, 3])

In [ ]:
df1

In [ ]:
pd.get_dummies(df1)

In [ ]:
df1

In [ ]:
na_df = pd.DataFrame([[1, 2, 3, 4, 5], [3, 4, 5, 1, np.nan],
                    [6, 4, 5, np.nan, np.nan], [1, 2, np.nan, np.nan, np.nan], 
                    [1, 7, np.nan, np.nan, np.nan], [1, 7, np.nan, np.nan, np.nan],
                    [1, 7, np.nan, np.nan, np.nan], [1, 7, np.nan, np.nan, np.nan],
                    [1, 7, np.nan, np.nan, np.nan], [1, 7, np.nan, np.nan, np.nan]],
                    columns=['key1','A','B','C','D'])
na_df

In [ ]:
na_df4 = na_df.copy(deep=True)
na_df4 = drop_rows_and_cols_with_NA_below_thresholds(na_df4, key_names=key_names, col_thresh=0.1, row_thresh=0.04)
na_df4

In [ ]:
na_df4 = na_df.copy(deep=True)
na_df4 = na_df4.dropna(axis=1, thresh=1) # droping NA columns
na_df4

In [ ]:
na_df2=na_df.copy(deep=True)
na_df2 = na_df2.dropna(axis=0, how='all',subset={'B','C','A'})
na_df2

In [ ]:
na_df_columns = list(na_df.columns)
na_df_columns

In [ ]:
na_df_columns_set=set(na_df_columns)
na_df_columns_set

In [ ]:
list(na_df_columns_set)

In [ ]:
set(na_df2)

In [ ]:
na_df3=na_df.copy(deep=True)
drop_NA_only_columns_and_rows(na_df3)

In [ ]:
[1,2,3] - [1,2]

In [ ]:
set([1,2,3]) - set([1,2,4])

In [ ]:
#old drop NA function v1
def drop_NA_only_columns_and_rows(input_df, key_names=key_names):
    df = input_df.copy(deep=True)
    df_columns = set(df)
    df_columns_without_keys = df_columns - set(key_names)
    df = df.dropna(axis=0, how='all', subset=df_columns_without_keys) # droping rows that have all NA values except for keys
    df = df.dropna(axis=1, how='all') # droping NA columns
    return df

In [ ]:
#old drop NA function v2
def drop_rows_with_NA_only_and_cols_with_NA_below_threshold(input_df, key_names=key_names, threshold_percent=0.20):
    df = input_df.copy(deep=True)
    
    df_columns = set(df)
    df_columns_without_keys = df_columns - set(key_names)
    df = df.dropna(axis=0, how='all', subset=df_columns_without_keys) # droping rows that have all NA values except for keys
    
    number_of_rows = len(df)
    threshold_integer = round(threshold_percent * number_of_rows)
    df = df.dropna(axis=1, thresh=threshold_integer) # droping columns that have non-NA cell count is below threshold
    return df

In [ ]:
#old with regex
#function to replace ',' with '.'
#import re
def replace_commas_with_dots_in_string(single_string):
    if type(single_string) == str:
        ##df = input_dataframe.copy(deep=True)
        ##regex_input = '^[0-9]+,[0-9]+$'
        ##regex_output = '^[0-9]+\.[0-9]+$'
        ##single_string = re.sub('^[0-9]+,[0-9]+$', '^[0-9]+\.[0-9]+$', single_string)
        ##df.replace(to_replace=regex_input, value=regex_output, regex=True)
        single_string = single_string.replace(',','.')
    return single_string

In [ ]:
dataset_full_clean

In [ ]:
list(dataset_full_clean.columns)

In [ ]:
col_names_no_key_no_target = get_col_names_without_target_and_keys(dataset_full_clean)

In [ ]:
dataset_full_clean[col_names_no_key_no_target]

In [ ]:
dataset_full_clean[col_names_no_key_no_target].shape

In [ ]:
dataset_full_clean.shape

In [ ]:
dataset_full_clean[col_names_no_key_no_target]

In [ ]:
col_names_no_key_no_target

In [ ]:
dataset_full_clean.dtypes

In [ ]:
dataset_full_clean['v4']

In [ ]:
dataset_full_clean['v12']

In [73]:
na_df = pd.DataFrame([[1.5, "2,5", 3, 4, 5], [3.5, 4.5, 5, 1, np.nan],
                    [6, "4,4", 5, "text with 4,5", "4,5 another text"], [1, 2, np.nan, np.nan, np.nan]],
                    columns=['key1','A','B','C','D'])
na_df

,key1,A,B,C,D
0,1.5,"2,5",3.0,4,5
1,3.5,4.5,5.0,1,NaN
2,6.0,"4,4",5.0,"text with 4,5","4,5 another text"
3,1.0,2,NaN,NaN,NaN


In [74]:
na_df = na_df.applymap(replace_commas_with_dots_in_string)
na_df

,key1,A,B,C,D
0,1.5,2.5,3.0,4,5
1,3.5,4.5,5.0,1,NaN
2,6.0,4.4,5.0,text with 4.5,4.5 another text
3,1.0,2,NaN,NaN,NaN


In [75]:
na_df.dtypes

key1    float64
A        object
B       float64
C        object
D        object
dtype: object

In [76]:
na_df = na_df.applymap(convert_floats_in_string_to_floats)
na_df

error
error


,key1,A,B,C,D
0,1.5,2.5,3.0,4,5
1,3.5,4.5,5.0,1,NaN
2,6.0,4.4,5.0,text with 4.5,4.5 another text
3,1.0,2.0,NaN,NaN,NaN


In [77]:
na_df.dtypes

key1    float64
A       float64
B       float64
C        object
D        object
dtype: object

In [ ]:
replace_commas_with_dots_in_df(na_df)

In [ ]:
na_df.replace(",",".")

In [108]:
na_df

,key1,A,B,C,D
0,1.5,2.5,3.0,4,5
1,3.5,4.5,5.0,1,NaN
2,6.0,4.4,5.0,text with 4.5,4.5 another text
3,1.0,2.0,NaN,NaN,NaN


In [114]:
na_df.drop('B', axis=1)

,key1,A,C,D
0,1.5,2.5,4,5
1,3.5,4.5,1,NaN
2,6.0,4.4,text with 4.5,4.5 another text
3,1.0,2.0,NaN,NaN


In [117]:
na_df = na_df.drop(['A','B'], axis=1)
na_df

ValueError: labels ['A' 'B'] not contained in axis

In [118]:
na_df

,key1,C,D
0,1.5,4,5
1,3.5,1,NaN
2,6.0,text with 4.5,4.5 another text
3,1.0,NaN,NaN


In [58]:
dataset_full.to_csv('output_dataset_full.csv')

In [142]:
dataset_full

,v001,v002,response,v4,v5,v14,v29,v120,v123,v173,...,v196,v197,v198,v199,v200,v201,v202,v203,v204,v172.1
0,15.0,35.0,0,1400.00,1.0,1400.00,1.0,NaN,NaN,1998-04-28 20:00:00,...,3.0,508.0,3.0,Moderate,311.0,Moderate By Proxy Reputation And Country Code,Good,No,business,U
1,13.0,21.0,1,500.00,1.0,614.85,1.0,NaN,NaN,1998-04-28 20:00:00,...,3.0,508.0,3.0,Moderate,311.0,Moderate By Proxy Reputation And Country Code,Good,No,residential,U
2,13.0,66.0,0,4975.86,2.0,4975.86,27.0,NaN,1.0,NaN,...,4.0,510.0,3.0,Moderate,311.0,Moderate By Proxy Reputation And Country Code,Good,No,residential,U
3,13.0,20.0,0,NaN,NaN,NaN,NaN,6.0,NaN,1999-12-22 19:00:00,...,3.0,508.0,3.0,Moderate,311.0,Moderate By Proxy Reputation And Country Code,Good,No,cellular,U
4,13.0,21.0,1,NaN,NaN,NaN,NaN,2.0,1.0,1995-08-13 07:00:00,...,3.0,508.0,3.0,Moderate,311.0,Moderate By Proxy Reputation And Country Code,Good,No,cellular,U
5,13.0,43.0,0,2500.00,1.0,3225.38,37.0,10.0,24.0,1999-10-07 20:00:00,...,3.0,508.0,3.0,Moderate,311.0,Moderate By Proxy Reputation And Country Code,Good,No,residential,U
6,14.0,27.0,1,13632.02,6.0,11675.93,38.0,4.0,6.0,1996-06-22 08:00:00,...,3.0,508.0,3.0,Moderate,311.0,Moderate By Proxy Reputation And Country Code,Good,No,residential,U
7,14.0,29.0,1,748.00,1.0,1277.33,5.0,3.0,NaN,1999-12-22 19:00:00,...,3.0,508.0,3.0,Moderate,311.0,Moderate By Proxy Reputation And Country Code,Good,No,residential,U
8,14.0,33.0,0,NaN,NaN,NaN,28.0,NaN,NaN,1995-08-13 07:00:00,...,3.0,508.0,3.0,Moderate,311.0,Moderate By Proxy Reputation And Country Code,Good,No,residential,U
9,14.0,21.0,0,NaN,NaN,NaN,5.0,1.0,4.0,1996-06-22 08:00:00,...,3.0,508.0,3.0,Moderate,311.0,Moderate By Proxy Reputation And Country Code,Good,No,residential,U


In [143]:
dataset_full.dtypes

v001        float64
v002        float64
response      int64
v4          float64
v5          float64
v14         float64
v29         float64
v120        float64
v123        float64
v173         object
v174        float64
v175         object
v176        float64
v177         object
v178         object
v179         object
v180        float64
v181        float64
v182        float64
v183        float64
v184         object
v185        float64
v186         object
v191        float64
v192        float64
v193         object
v194         object
v195         object
v196        float64
v197        float64
v198        float64
v199         object
v200        float64
v201         object
v202         object
v203         object
v204         object
v172.1       object
dtype: object

0          1400
1           500
2       4975.86
3           NaN
4           NaN
5          2500
6      13632.02
7           748
8           NaN
9           NaN
10         3000
11     49813.88
12         3000
13          NaN
14          NaN
15     50874.66
17         3300
18      1736.45
20         5700
21          150
22          300
23          NaN
24         7500
25         1750
26         1600
27          600
28         2350
29          NaN
30         1800
32          NaN
         ...   
768        1000
769         NaN
770        1200
771        2500
772         NaN
773        6000
774        3000
775        2100
776     7758.09
777        1000
778         160
779         NaN
780        3000
781        3200
782         320
783        1150
784         NaN
785        1000
786      759.32
787        2000
788         NaN
789         NaN
790         100
791        5886
792         NaN
793         NaN
794        3000
795        4800
796         300
797        1700
Name: v4, Length: 779, d

In [88]:
int(5.5)

5

In [121]:
dataset_full_not_cleaned

,key1,v001,v002,key2,response,v3,v4,v5,v6,v7,...,v197,v198,v199,v200,v201,v202,v203,v204,v171.1,v172.1
0,4,15,35,4,0,NaN,1400,1.0,NaN,NaN,...,508.0,3.0,Moderate,311.0,Moderate By Proxy Reputation And Country Code,Good,No,business,"0,1",U
1,88,13,21,34,1,NaN,500,1.0,NaN,NaN,...,508.0,3.0,Moderate,311.0,Moderate By Proxy Reputation And Country Code,Good,No,residential,"0,1",U
2,139,13,66,808,0,NaN,"4975,86",2.0,NaN,NaN,...,510.0,3.0,Moderate,311.0,Moderate By Proxy Reputation And Country Code,Good,No,residential,"0,1",U
3,148,13,20,65,0,NaN,NaN,NaN,NaN,NaN,...,508.0,3.0,Moderate,311.0,Moderate By Proxy Reputation And Country Code,Good,No,cellular,"0,1",U
4,159,13,21,312,1,NaN,NaN,NaN,NaN,NaN,...,508.0,3.0,Moderate,311.0,Moderate By Proxy Reputation And Country Code,Good,No,cellular,"0,1",U
5,162,13,43,6218,0,NaN,2500,1.0,NaN,NaN,...,508.0,3.0,Moderate,311.0,Moderate By Proxy Reputation And Country Code,Good,No,residential,"0,1",U
6,175,14,27,241,1,NaN,"13632,02",6.0,NaN,NaN,...,508.0,3.0,Moderate,311.0,Moderate By Proxy Reputation And Country Code,Good,No,residential,"0,1",U
7,215,14,29,107,1,NaN,748,1.0,NaN,NaN,...,508.0,3.0,Moderate,311.0,Moderate By Proxy Reputation And Country Code,Good,No,residential,"0,1",U
8,246,14,33,102,0,NaN,NaN,NaN,NaN,NaN,...,508.0,3.0,Moderate,311.0,Moderate By Proxy Reputation And Country Code,Good,No,residential,"1,7",U
9,280,14,21,216,0,NaN,NaN,NaN,NaN,NaN,...,508.0,3.0,Moderate,311.0,Moderate By Proxy Reputation And Country Code,Good,No,residential,"0,1",U
